In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle
import requests
from bs4 import BeautifulSoup
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import datetime
import os
import shutil

if not os.path.exists("models/savedModels"):
    os.makedirs("models/savedModels")
else:
    shutil.rmtree("models/savedModels")
    os.makedirs("models/savedModels")

if not os.path.exists("output/linearRegression/linearRegression"):
    os.makedirs("output/linearRegression")
else:
    shutil.rmtree("output/linearRegression")
    os.makedirs("output/linearRegression")

if not os.path.exists("models/logs/linearRegression"):
    os.makedirs("models/logs/linearRegression")
if not os.path.exists("prediction/logs/linearRegression"):
    os.makedirs("prediction/logs/linearRegression")

In [14]:
#Introducción
print("***************Introducción*******************")
print("Se plantea hacer una predicción de los precios historicos del S&P500, para que los operadores tengan una referencia de precio de cierre, basada en datos históricos")
print("Los datos se obtendrán de la api de yahoo finance, para no tener que ir cargando ficheros de dataset")
print("Tras entrenar el modelo se hará un guardado del modelo para poder utilizarlo agilmente en el futuro")
print("Se ejecutará una predicción para el día con los modelos entrenados")

***************Introducción*******************
Se plantea hacer una predicción de los precios historicos del S&P500, para que los operadores tengan una referencia de precio de cierre, basada en datos históricos
Los datos se obtendrán de la api de yahoo finance, para no tener que ir cargando ficheros de dataset
Tras entrenar el modelo se hará un guardado del modelo para poder utilizarlo agilmente en el futuro
Se ejecutará una predicción para el día con los modelos entrenados


In [15]:
# Fetch the list of tickers from the Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find("table", {"class": "wikitable sortable"})
tickers = []
for row in table.findAll("tr")[1:]:
    ticker = row.findAll("td")[0].text.strip()
    tickers.append(ticker)

print(tickers)

C:\Users\aitor\anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BALL', 'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'D

In [16]:
# Fetch historical data for each ticker
for ticker in tickers:
    nowDate = datetime.datetime.now()
    try:
        try:
            if "." in ticker:          #Yahoo Finance uses - instead of . (ex. BRK.B -> BRK-B)
                ticker = ticker.split(".")
                ticker = ticker[0]+ "-" + ticker[1]
            data = yf.download(ticker)

            # Clean and preprocess the data
            dataClose = data[["Close"]]
            dataClose = dataClose.rolling(window=10).mean()
            dataClose.dropna(inplace=True)
            dataClose.reset_index(drop=True, inplace=True)

            data = data.rolling(window=10).mean()
            data.dropna(inplace=True)
            data.reset_index(drop=True, inplace=True)

            # Create a dataset for training the model
            X = data.drop("Close", axis=1)
            y = dataClose["Close"]

        except ValueError as e:
            print(f"Couldn't fetch data for {ticker}")
            with open(f"models/logs/{ticker}.txt", "a") as f:
                    f.write(f"[{nowDate}]{ticker} : {e}\n")
            continue
        

        # Handling missing values
        if np.any(np.isnan(X)):
            X = X.fillna(X.mean())
        if np.any(np.isnan(y)):
            y = y.fillna(y.mean())


        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Create a linear regression model
        model = LinearRegression()

        # Train the model
        model.fit(X_train, y_train)
        print(f'Model for {ticker} has been trained')
        
         # Make predictions on the test set
        y_pred = model.predict(X_test)

        # Evaluate the model
        mae = mean_absolute_error(y_test, y_pred)
        print(f"Mean Absolute Error for {ticker}: {mae}")
        mse = mean_squared_error(y_test, y_pred)
        print(f"Mean Squared Error for {ticker}: {mse}")
        r2 = r2_score(y_test, y_pred)
        print(f"R^2 score for {ticker}: {r2}")

         # Saving the Model (serialized)
        with open(f'models/savedModels/{ticker}.pickle','wb') as f:
            pickle.dump(model,f)

       # Saving the predictions to a file
        with open("output/model_outputs.txt", "a") as f:
            for i in range(len(y_pred)):
                f.write(f"{ticker} : {y_pred[i]}\n")
                
    except ValueError as e:
        print(f"Couldn't fetch data for {ticker}")
        with open(f"models/logs/{ticker}.txt", "a") as f:
                f.write(f"[{nowDate}]{ticker} : {e}\n")    

[*********************100%***********************]  1 of 1 completed
Model for MMM has been trained
Mean Absolute Error for MMM: 0.0844007313398852
Mean Squared Error for MMM: 0.024681657071011966
R^2 score for MMM: 0.9999927419922997
[*********************100%***********************]  1 of 1 completed
Model for AOS has been trained
Mean Absolute Error for AOS: 0.03162101870977408
Mean Squared Error for AOS: 0.003798563478187235
R^2 score for AOS: 0.9999910539332026
[*********************100%***********************]  1 of 1 completed
Model for ABT has been trained
Mean Absolute Error for ABT: 0.044728385288951636
Mean Squared Error for ABT: 0.005589661345168081
R^2 score for ABT: 0.9999938899254743
[*********************100%***********************]  1 of 1 completed
Model for ABBV has been trained
Mean Absolute Error for ABBV: 0.12042472332241033
Mean Squared Error for ABBV: 0.02372048707398923
R^2 score for ABBV: 0.9999753979219188
[*********************100%***********************]  1

[*********************100%***********************]  1 of 1 completed
Model for AMP has been trained
Mean Absolute Error for AMP: 0.18259866365178987
Mean Squared Error for AMP: 0.07874820406111845
R^2 score for AMP: 0.9999858229657266
[*********************100%***********************]  1 of 1 completed
Model for ABC has been trained
Mean Absolute Error for ABC: 0.07471150664212975
Mean Squared Error for ABC: 0.01656989954776377
R^2 score for ABC: 0.9999899005859513
[*********************100%***********************]  1 of 1 completed
Model for AME has been trained
Mean Absolute Error for AME: 0.033253728133706424
Mean Squared Error for AME: 0.004180756986805274
R^2 score for AME: 0.9999967346874259
[*********************100%***********************]  1 of 1 completed
Model for AMGN has been trained
Mean Absolute Error for AMGN: 0.11538238475328554
Mean Squared Error for AMGN: 0.04217645475077328
R^2 score for AMGN: 0.999992071916243
[*********************100%***********************]  1 o

[*********************100%***********************]  1 of 1 completed
Model for BK has been trained
Mean Absolute Error for BK: 0.039205272594365806
Mean Squared Error for BK: 0.005491434546665141
R^2 score for BK: 0.9999838313516942
[*********************100%***********************]  1 of 1 completed
Model for BA has been trained
Mean Absolute Error for BA: 0.0889795759772031
Mean Squared Error for BA: 0.0405257938729431
R^2 score for BA: 0.9999938076752872
[*********************100%***********************]  1 of 1 completed
Model for BKNG has been trained
Mean Absolute Error for BKNG: 1.882612312729513e-13
Mean Squared Error for BKNG: 5.731971106657202e-26
R^2 score for BKNG: 1.0
[*********************100%***********************]  1 of 1 completed
Model for BWA has been trained
Mean Absolute Error for BWA: 0.04280126247236836
Mean Squared Error for BWA: 0.004658729237791714
R^2 score for BWA: 0.9999852165837935
[*********************100%***********************]  1 of 1 completed
Model

[*********************100%***********************]  1 of 1 completed
Model for CMG has been trained
Mean Absolute Error for CMG: 9.80756493671298e-13
Mean Squared Error for CMG: 1.670641013959888e-24
R^2 score for CMG: 1.0
[*********************100%***********************]  1 of 1 completed
Model for CB has been trained
Mean Absolute Error for CB: 0.10378321777491395
Mean Squared Error for CB: 0.028350800424398807
R^2 score for CB: 0.9999896069305008
[*********************100%***********************]  1 of 1 completed
Model for CHD has been trained
Mean Absolute Error for CHD: 0.03278719469298409
Mean Squared Error for CHD: 0.004440858695226028
R^2 score for CHD: 0.9999933571528689
[*********************100%***********************]  1 of 1 completed
Model for CI has been trained
Mean Absolute Error for CI: 0.09395941874712523
Mean Squared Error for CI: 0.03476173998211301
R^2 score for CI: 0.9999931715491489
[*********************100%***********************]  1 of 1 completed
Model for

[*********************100%***********************]  1 of 1 completed
Model for DRI has been trained
Mean Absolute Error for DRI: 0.07980472508795766
Mean Squared Error for DRI: 0.01668834841338924
R^2 score for DRI: 0.9999889854485567
[*********************100%***********************]  1 of 1 completed
Model for DVA has been trained
Mean Absolute Error for DVA: 5.217967295692709e-14
Mean Squared Error for DVA: 6.247429523357454e-27
R^2 score for DVA: 1.0
[*********************100%***********************]  1 of 1 completed
Model for DE has been trained
Mean Absolute Error for DE: 0.0813483492546491
Mean Squared Error for DE: 0.03181859191798365
R^2 score for DE: 0.9999946857901977
[*********************100%***********************]  1 of 1 completed
Model for DAL has been trained
Mean Absolute Error for DAL: 0.06756496280131735
Mean Squared Error for DAL: 0.008202904668725787
R^2 score for DAL: 0.9999730912678326
[*********************100%***********************]  1 of 1 completed
Model 

...

[*********************100%***********************]  1 of 1 completed
Model for NXPI has been trained
Mean Absolute Error for NXPI: 0.15814346382739927
Mean Squared Error for NXPI: 0.05671404113578332
R^2 score for NXPI: 0.999980888869213
[*********************100%***********************]  1 of 1 completed
Model for ORLY has been trained
Mean Absolute Error for ORLY: 2.76152219170597e-10
Mean Squared Error for ORLY: 1.5867244065555084e-19
R^2 score for ORLY: 1.0
[*********************100%***********************]  1 of 1 completed
Model for OXY has been trained
Mean Absolute Error for OXY: 0.06441570957055173
Mean Squared Error for OXY: 0.011924311931557513
R^2 score for OXY: 0.9999860573521229
[*********************100%***********************]  1 of 1 completed
Model for ODFL has been trained
Mean Absolute Error for ODFL: 0.0891661268440211
Mean Squared Error for ODFL: 0.028478529140816018
R^2 score for ODFL: 0.9999944802322089
[*********************100%***********************]  1 of 1 

[*********************100%***********************]  1 of 1 completed
Model for PEG has been trained
Mean Absolute Error for PEG: 0.03700759563237754
Mean Squared Error for PEG: 0.0029753189715907304
R^2 score for PEG: 0.999989386744824
[*********************100%***********************]  1 of 1 completed
Model for PTC has been trained
Mean Absolute Error for PTC: 4.9625639592331775e-14
Mean Squared Error for PTC: 7.155261664672375e-27
R^2 score for PTC: 1.0
[*********************100%***********************]  1 of 1 completed
Model for PSA has been trained
Mean Absolute Error for PSA: 0.14274583430212492
Mean Squared Error for PSA: 0.057132841738328034
R^2 score for PSA: 0.9999930883507071
[*********************100%***********************]  1 of 1 completed
Model for PHM has been trained
Mean Absolute Error for PHM: 0.03661025543322678
Mean Squared Error for PHM: 0.004046936161664943
R^2 score for PHM: 0.9999777361660708
[*********************100%***********************]  1 of 1 complete

[*********************100%***********************]  1 of 1 completed
Model for SJM has been trained
Mean Absolute Error for SJM: 0.1068494384867264
Mean Squared Error for SJM: 0.025745137444376415
R^2 score for SJM: 0.9999844200705967
[*********************100%***********************]  1 of 1 completed
Model for SNA has been trained
Mean Absolute Error for SNA: 0.09655751475969544
Mean Squared Error for SNA: 0.026981836340498125
R^2 score for SNA: 0.99999230681287
[*********************100%***********************]  1 of 1 completed
Model for SEDG has been trained
Mean Absolute Error for SEDG: 4.3958239017873764e-11
Mean Squared Error for SEDG: 3.1472358721333217e-21
R^2 score for SEDG: 1.0
[*********************100%***********************]  1 of 1 completed
Model for SO has been trained
Mean Absolute Error for SO: 0.03354067747681587
Mean Squared Error for SO: 0.0024279799792248456
R^2 score for SO: 0.9999926447490798
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed
Model for TFC has been trained
Mean Absolute Error for TFC: 0.05248231249311807
Mean Squared Error for TFC: 0.00652599676638272
R^2 score for TFC: 0.9999771515725175
[*********************100%***********************]  1 of 1 completed
Model for TYL has been trained
Mean Absolute Error for TYL: 0.08474899816287983
Mean Squared Error for TYL: 0.04299249746252811
R^2 score for TYL: 0.9999963768183849
[*********************100%***********************]  1 of 1 completed
Model for TSN has been trained
Mean Absolute Error for TSN: 0.056131803660079645
Mean Squared Error for TSN: 0.007376061492375774
R^2 score for TSN: 0.9999867743149838
[*********************100%***********************]  1 of 1 completed
Model for USB has been trained
Mean Absolute Error for USB: 0.031832217645219506
Mean Squared Error for USB: 0.002850542592022394
R^2 score for USB: 0.9999912942662211
[*********************100%***********************]  1 of

[*********************100%***********************]  1 of 1 completed
Model for WY has been trained
Mean Absolute Error for WY: 0.0682845988571887
Mean Squared Error for WY: 0.009346864757444609
R^2 score for WY: 0.9999590338098616
[*********************100%***********************]  1 of 1 completed
Model for WHR has been trained
Mean Absolute Error for WHR: 0.14563901274477437
Mean Squared Error for WHR: 0.05172649164591486
R^2 score for WHR: 0.9999826703458686
[*********************100%***********************]  1 of 1 completed
Model for WMB has been trained
Mean Absolute Error for WMB: 0.03489507451803556
Mean Squared Error for WMB: 0.0030923453281719664
R^2 score for WMB: 0.9999804259285475
[*********************100%***********************]  1 of 1 completed
Model for WTW has been trained
Mean Absolute Error for WTW: 0.19063478202933554
Mean Squared Error for WTW: 0.067186477688439
R^2 score for WTW: 0.9999730198365867
[*********************100%***********************]  1 of 1 compl

In [17]:
# load the trained model
try:
    with open(f'models/savedModels/{ticker}.pickle','rb') as f:
        model = pickle.load(f)
except FileNotFoundError as e:
    print(f"Couldn't load model for: {ticker}")
    with open(f"prediction/logs/{ticker}.txt", "a") as f:
        for i in range(len(y_pred)):
            f.write(f"[{nowDate}]{ticker} : {e}\n")

# Get the data from yesterday
today = datetime.datetime.today()
yesterday = today - datetime.timedelta(days=1)

if os.path.exists(f"{today.date()}.txt"):
    shutil.rmtree(f"{today.date()}.txt")

# Fetch historical data for each ticker
for ticker in tickers:
    try:
        if "." in ticker:          #Yahoo Finance uses - instead of . (ex. BRK.B -> BRK-B)
            ticker = ticker.split(".")
            ticker = ticker[0]+ "-" + ticker[1]
        data = yf.download(ticker, start=yesterday, end=today)
        print(data)
        # Preprocess the data
        data= data.drop("Close", axis=1)
      
        # Make a prediction for today's close
        prediction = model.predict(data)
        print(f'Prediction for {ticker} has been done')
        
        # Save the prediction to a file
        with open(f'Predictions {today.date()}.txt', 'a') as f:
            f.write(f'{ticker}: {prediction}\n')
    except ValueError as e:
        print(f"Couldn't fetch data for {ticker}")
        with open(f"prediction/logs/{ticker}.txt", "a") as f:
            for i in range(len(y_pred)):
                f.write(f"[{nowDate}]{ticker} : {e}\n")
        continue


[*********************100%***********************]  1 of 1 completed
              Open        High         Low       Close   Adj Close   Volume
Date                                                                       
2023-02-07  115.82  117.209999  114.769997  116.900002  116.900002  2182100
Prediction for MMM has been done
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2023-02-07  66.900002  67.150002  65.760002  66.629997  66.629997  1197800
Prediction for AOS has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-02-07  110.040001  112.410004  109.879997  111.989998  111.989998   

             Volume  
Date                 
2023-02

[*********************100%***********************]  1 of 1 completed
                 Open       High    Low      Close  Adj Close   Volume
Date                                                                  
2023-02-07  46.689999  46.810001  46.25  46.560001  46.560001  6917400
Prediction for MO has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High        Low       Close   Adj Close  \
Date                                                                    
2023-02-07  101.169998  102.410004  98.080002  102.110001  102.110001   

               Volume  
Date                   
2023-02-07  119384500  
Prediction for AMZN has been done
[*********************100%***********************]  1 of 1 completed
             Open   High    Low  Close  Adj Close   Volume
Date                                                      
2023-02-07  11.88  11.93  11.71  11.89      11.89  7373500
Prediction for AMCR has been done
[**********

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-02-07  129.520004  135.740005  129.009995  134.910004  134.910004   

             Volume  
Date                 
2023-02-07  2299300  
Prediction for ANET has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2023-02-07  193.410004  198.029999  193.220001  197.470001  197.470001  670300
Prediction for AJG has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2023-02-07  132.539993  136.199997  131.53

[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2023-02-07  46.740002  47.630001  46.619999  47.310001  47.310001  2653500
Prediction for BWA has been done
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low  Close  Adj Close   Volume
Date                                                                  
2023-02-07  72.639999  74.080002  72.120003   73.5       73.5  1215900
Prediction for BXP has been done
[*********************100%***********************]  1 of 1 completed
             Open   High        Low  Close  Adj Close   Volume
Date                                                          
2023-02-07  47.07  48.27  47.049999  48.09      48.09  5284000
Prediction for BSX has been done
[*********************100%***********************]  1 of 1 complete

[*********************100%***********************]  1 of 1 completed
                 Open   High    Low      Close  Adj Close   Volume
Date                                                              
2023-02-07  29.139999  29.67  29.07  29.610001  29.610001  4192600
Prediction for CNP has been done
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2023-02-07  73.620003  75.160004  72.370003  74.599998  74.599998  1465600
Prediction for CDAY has been done
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low  Close  Adj Close   Volume
Date                                                                  
2023-02-07  86.080002  86.459999  82.769997  84.25      84.25  2990800
Prediction for CF has been done
[*********************100%***********************]  1 o

...

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2023-02-07  120.209999  122.519997  119.400002  122.300003  122.300003  929800
Prediction for RL has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2023-02-07  111.529999  114.290001  111.459999  113.919998  113.919998  831600
Prediction for RJF has been done
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2023-02-07  97.160004  98.199997  95.800003  97.870003  97.870003  3814500
Prediction for RTX has been d

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-02-07  127.849998  127.849998  123.209999  125.900002  125.900002   

             Volume  
Date                 
2023-02-07  2442800  
Prediction for SPG has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-02-07  114.669998  123.690002  113.480003  122.970001  122.970001   

             Volume  
Date                 
2023-02-07  5741700  
Prediction for SWKS has been done
[*********************100%***********************]  1 of 1 completed
             Open        High         Low  Close  Adj Close  Volume
Date                                                               
2023-02-07  

[*********************100%***********************]  1 of 1 completed
                  Open       High         Low       Close   Adj Close  Volume
Date                                                                         
2023-02-07  431.040009  439.48999  429.920013  437.390015  437.390015  154900
Prediction for TDY has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low  Close  Adj Close  Volume
Date                                                                    
2023-02-07  251.580002  257.640015  249.869995  257.5      257.5  185400
Prediction for TFX has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-02-07  106.919998  110.889999  106.809998  110.199997  110.199997   

             Volume  
Date                 
2023-02

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2023-02-07  144.259995  145.289993  142.440002  144.509995  144.509995  629800
Prediction for UHS has been done
[*********************100%***********************]  1 of 1 completed
             Open        High         Low       Close   Adj Close   Volume
Date                                                                      
2023-02-07  129.0  135.300003  128.740005  134.520004  134.520004  3589900
Prediction for VLO has been done
[*********************100%***********************]  1 of 1 completed
                 Open  High        Low  Close  Adj Close   Volume
Date                                                             
2023-02-07  51.619999  52.0  51.049999   51.5       51.5  2862300
Prediction for VTR has been done
[*********************100%********

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2023-02-07  148.869995  150.199997  147.300003  149.619995  149.619995  647200
Prediction for WHR has been done
[*********************100%***********************]  1 of 1 completed
             Open   High    Low      Close  Adj Close   Volume
Date                                                          
2023-02-07  31.74  32.23  31.43  32.110001  32.110001  8534500
Prediction for WMB has been done
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2023-02-07  253.009995  256.559998  251.729996  256.179993  256.179993  548500
Prediction for WTW has been done
[*********************100%*****

In [18]:
#Resultados
print("***************Resultados*******************")
print("Se ha conseguido hacer una aproximación de precio de cierre basada en los precios históricos de la acción, para posibles trader como solamente un posible indicio del posible precio de cierre")
print("Nos permite conocer como debe poder cerrar la acción de las compañías basadas en su volatilidad, lo que en dias no muy volátiles puede ser muy útil")

***************Resultados*******************
Se ha conseguido hacer una aproximación de precio de cierre basada en los precios históricos de la acción
Nos permite conocer como debe poder cerrar la acción de las compañías basadas en su volatilidad, lo que en dias no muy volátiles puede ser muy útil


In [19]:
#Mejoras
print("***************Mejoras propuestas*******************")
print("Obviamente el precio que obtenemos no tiene en cuenta eventos como presentacion de resultados, dividendos, etc. Sería un buen punto añadido")
print("Actualmente al descargar los datos de manera dinámica se requiere de acceso a internet, y la api de Yahoo finance puede estar caida, lo que desemboca problemas")
print("Dar soporte a mas ínidces bursátiles y acciones en particular")

***************Mejoras propuestas*******************
Obviamente el precio que obtenemos no tiene en cuenta eventos como presentacion de resultados, dividendos, etc. Sería un buen punto añadido
Actualmente al descargar los datos de manera dinámica se requiere de acceso a internet, y la api de Yahoo finance puede estar caida, lo que desemboca problemas
Dar soporte a mas ínidces bursátiles y acciones en particular
